In [1]:
import pandas as pd

In [6]:
df=pd.read_csv('store_zara.csv')

In [7]:
df.head()

,brand,url,sku,name,description,price,currency,images,scraped_at,terms,section,error,image_downloads
0,Zara,https://www.zara.com/us/en/basic-puffer-jacket...,272145190-250-2,BASIC PUFFER JACKET,Puffer jacket made of tear-resistant ripstop f...,19.99,USD,['https://static.zara.net/photos///2023/I/0/2/...,2024-02-19T08:50:05.654618,jackets,MAN,NaN,"['e8e4ae57-8b72-44ff-aa5d-84de3ed37d9e', '0cb3..."
1,Zara,https://www.zara.com/us/en/tuxedo-jacket-p0889...,324052738-800-46,TUXEDO JACKET,Straight fit blazer. Pointed lapel collar and ...,169.00,USD,['https://static.zara.net/photos///2024/V/0/1/...,2024-02-19T08:50:06.590930,jackets,MAN,NaN,"['b42b0725-cfe2-4748-af3e-3abe590d83cd', 'ed9c..."
2,Zara,https://www.zara.com/us/en/slim-fit-suit-jacke...,335342680-800-44,SLIM FIT SUIT JACKET,Slim fit jacket. Notched lapel collar. Long sl...,129.00,USD,['https://static.zara.net/photos///2023/I/0/2/...,2024-02-19T08:50:07.301419,jackets,MAN,NaN,"['c27bdddf-2f9c-4693-976f-0d1272212c12', '07e6..."
3,Zara,https://www.zara.com/us/en/stretch-suit-jacket...,328303236-420-44,STRETCH SUIT JACKET,Slim fit jacket made of viscose blend fabric. ...,129.00,USD,['https://static.zara.net/photos///2024/V/0/1/...,2024-02-19T08:50:07.882922,jackets,MAN,NaN,"['0cbc84c8-f9ff-4f02-940c-ff642b5af379', 'dd47..."
4,Zara,https://www.zara.com/us/en/double-faced-jacket...,312368260-800-2,DOUBLE FACED JACKET,Jacket made of faux leather faux shearling wit...,139.00,USD,['https://static.zara.net/photos///2024/V/0/2/...,2024-02-19T08:50:08.453847,jackets,MAN,NaN,"['1ccc9c87-49c9-4825-9d67-4c1f61e5db1f', '1349..."


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3129 entries, 0 to 3128
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   brand            3129 non-null   object 
 1   url              3129 non-null   object 
 2   sku              3129 non-null   object 
 3   name             3065 non-null   object 
 4   description      3059 non-null   object 
 5   price            3129 non-null   float64
 6   currency         3129 non-null   object 
 7   images           3129 non-null   object 
 8   scraped_at       3129 non-null   object 
 9   terms            3129 non-null   object 
 10  section          3129 non-null   object 
 11  error            0 non-null      float64
 12  image_downloads  3129 non-null   object 
dtypes: float64(2), object(11)
memory usage: 317.9+ KB


In [14]:
import os
# SUBSTITUA PELO SEU CAMINHO CONFIRMADO!
PASTA_TESTE = 'D:\Mestrado\shoppingGPT\shoppinggpt\images'

if os.path.exists(PASTA_TESTE):
    print(f"SUCESSO! O caminho '{PASTA_TESTE}' existe.")
    # Tenta listar um ficheiro para verificar se as imagens estão lá
    print("Primeiros 5 itens na pasta:", os.listdir(PASTA_TESTE)[:5])
else:
    print(f"ERRO! O caminho '{PASTA_TESTE}' NÃO existe. Verifique a localização.")

SUCESSO! O caminho 'D:\Mestrado\shoppingGPT\shoppinggpt\images' existe.
Primeiros 5 itens na pasta: ['imagens_zara']


In [15]:
import pandas as pd
import os
import ast
import numpy as np

# 🚨 CAMINHO CORRIGIDO: Este é o caminho final que funcionou (adicionando 'imagens_zara')
PASTA_IMAGENS_LOCAL = os.path.join(r'D:\Mestrado\shoppingGPT\shoppinggpt\images', 'imagens_zara')

# --- 1. FUNÇÕES DE PROCESSAMENTO ---

def processar_ids_imagem_corrigido(ids_string):
    """Converte a string da coluna 'image_downloads' para uma lista de IDs de ficheiro."""
    if pd.isna(ids_string):
        return []
    try:
        # ast.literal_eval converte a string ['id1', 'id2', ...] para uma lista Python
        lista_ids = ast.literal_eval(ids_string)
        return lista_ids if isinstance(lista_ids, list) else []
    except (ValueError, SyntaxError):
        return []

def get_caminho_local_imagem_principal(row):
    """
    Cria o caminho local para a primeira imagem do produto
    e verifica se o ficheiro existe no disco.
    """
    # Verifica se existem IDs
    if row['ids_ficheiro_limpos']:
        # Pega o primeiro ID da lista e adiciona a extensão .jpg
        filename = row['ids_ficheiro_limpos'][0].strip() + '.jpg'
        
        # Cria o caminho completo
        caminho_completo = os.path.join(PASTA_IMAGENS_LOCAL, filename)
        
        # Verifica se o ficheiro existe realmente no disco antes de retornar
        if os.path.exists(caminho_completo):
            return caminho_completo
        else:
            return None # Ficheiro não encontrado (problema no download)
            
    return None # Não há IDs de ficheiro

# --- 2. APLICAÇÃO AO DATAFRAME ---

print("A processar IDs e a criar a coluna de caminhos...")

# 2.1 Limpar/Criar a coluna de IDs
df['ids_ficheiro_limpos'] = df['image_downloads'].apply(processar_ids_imagem_corrigido)

# 2.2 Criar a coluna de Caminho Local
df['caminho_imagem_local'] = df.apply(get_caminho_local_imagem_principal, axis=1)

# --- 3. VERIFICAÇÃO FINAL ---

# Filtra apenas os produtos onde conseguimos encontrar a imagem localmente
df_pronto = df.dropna(subset=['caminho_imagem_local'])
percentagem_pronta = (len(df_pronto) / len(df)) * 100

print("\n--- Preparação de Dados Concluída ---")
print(f"Total de Produtos no CSV: {len(df)}")
print(f"Produtos prontos para LVM (Imagem Encontrada): {len(df_pronto)}")
print(f"Percentagem de sucesso no link local: {percentagem_pronta:.2f}%")

print("\nPrimeiros 5 Produtos (Verifique a coluna 'caminho_imagem_local'):")
print(df_pronto[['name', 'caminho_imagem_local']].head())

A processar IDs e a criar a coluna de caminhos...

--- Preparação de Dados Concluída ---
Total de Produtos no CSV: 3129
Produtos prontos para LVM (Imagem Encontrada): 3129
Percentagem de sucesso no link local: 100.00%

Primeiros 5 Produtos (Verifique a coluna 'caminho_imagem_local'):
                   name                               caminho_imagem_local
0   BASIC PUFFER JACKET  D:\Mestrado\shoppingGPT\shoppinggpt\images\ima...
1         TUXEDO JACKET  D:\Mestrado\shoppingGPT\shoppinggpt\images\ima...
2  SLIM FIT SUIT JACKET  D:\Mestrado\shoppingGPT\shoppinggpt\images\ima...
3   STRETCH SUIT JACKET  D:\Mestrado\shoppingGPT\shoppinggpt\images\ima...
4   DOUBLE FACED JACKET  D:\Mestrado\shoppingGPT\shoppinggpt\images\ima...


In [16]:
df_pronto.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3129 entries, 0 to 3128
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   brand                 3129 non-null   object 
 1   url                   3129 non-null   object 
 2   sku                   3129 non-null   object 
 3   name                  3065 non-null   object 
 4   description           3059 non-null   object 
 5   price                 3129 non-null   float64
 6   currency              3129 non-null   object 
 7   images                3129 non-null   object 
 8   scraped_at            3129 non-null   object 
 9   terms                 3129 non-null   object 
 10  section               3129 non-null   object 
 11  error                 0 non-null      float64
 12  image_downloads       3129 non-null   object 
 13  ids_ficheiro_limpos   3129 non-null   object 
 14  caminho_imagem_local  3129 non-null   object 
dtypes: float64(2), object

In [18]:
df_pronto.to_excel("dataset.xlsx", index=False)